In [1]:
#import dependencies
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
import pandas as pd
import os
# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float,ForeignKey 

import pymysql
pymysql.install_as_MySQLdb()

In [4]:
data_df=pd.read_csv("Cleaned_Data.csv",low_memory=False)

data_df=data_df.rename(columns={"Price Maximum": "PriceMaximum", "Price Minimum": "PriceMinimum","Start Date":"StartDate"})
#data_df=data_df.drop(['Unnamed: 10'], axis=1)
data_df.head()

,Genre,Latitude,Longitude,Name,PriceMaximum,PriceMinimum,StartDate,Subgenre,Venue,VenueID
0,Rock,41.506036,-81.699570,Taylor Swift reputation Stadium Tour,495.5,45.5,7/17/18,Pop,"""FirstEnergy Stadium, Home of the Cleveland Br...",KovZpZAFdEJA
1,Metal,41.542609,-87.771175,Ozzy Osbourne: No More Tours 2,250.0,29.5,9/21/18,Heavy Metal,"""Hollywood Casino Amphitheatre - Chicago, IL""",KovZpZAEktFA
2,Rock,41.542609,-87.771175,Lynyrd Skynyrd: Last of the Street Survivors F...,199.5,29.5,8/3/18,Pop,"""Hollywood Casino Amphitheatre - Chicago, IL""",KovZpZAEktFA
3,Rock,41.542609,-87.771175,Imagine Dragons: EVOLVE TOUR,146.0,35.0,7/13/18,Pop,"""Hollywood Casino Amphitheatre - Chicago, IL""",KovZpZAEktFA
4,Pop,41.542609,-87.771175,The Adventures of Kesha and Macklemore,137.0,30.5,7/14/18,Pop,"""Hollywood Casino Amphitheatre - Chicago, IL""",KovZpZAEktFA


In [5]:
#declare the base
Base=declarative_base()

#declare classes and attributes
class Events(Base):
    
    __tablename__="Events"
    id=Column(Integer,primary_key=True)
    genre=Column(String(255))
    Latitude = Column(Float)
    Longitude = Column(Float)
    Name=Column(String(255))
    PriceMaximum = Column(Float)
    PriceMinimum = Column(Float)
    
    StartDate=Column(String(255))
    Subgenre = Column(String(255))
    Venue = Column(String(255))
    VenueID = Column(String(255))
    
   

In [6]:
engine = create_engine(os.path.join("sqlite:///EventsDB.db"))
conn = engine.connect()
Base.metadata.create_all(conn)
session = Session(bind=engine)

In [7]:
data_df.to_sql('Events',conn,if_exists = 'append',index = False)
session.commit()

conn.close()

In [2]:
engine = create_engine(os.path.join("sqlite:///EventsDB.db"),echo=False)


conn = engine.connect()

In [3]:
Base = automap_base()
# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)
# Print all of the classes mapped to the Base
Base.classes.keys()

['Events']

In [4]:
events_db = Base.classes.Events

session = Session(engine)

In [5]:
first_row = session.query(events_db).first()
#first_row.__dict__

In [6]:
result_sample = session.query(events_db.Latitude,events_db.Longitude,events_db.Name,events_db.PriceMaximum,\
                       events_db.PriceMinimum,events_db.StartDate,events_db.Subgenre,events_db.Venue).all()

In [7]:
#result_sample[0]

In [8]:
result_list=[]
result_dict={}
names = ["Latitude", "Longitude", "Name","PriceMaximum","PriceMinimum","StartDate","Subgenre","Venue"]
for each in result_sample:
    result_dict={}
    for i in range(len(names)):
        result_dict[names[i]]=each[i]
        #print(each[1])
    result_list.append(result_dict)

In [22]:
result_list

[{'Latitude': 41.50603554,
  'Longitude': -81.69956958,
  'Name': 'Taylor Swift reputation Stadium Tour',
  'PriceMaximum': 495.5,
  'PriceMinimum': 45.5,
  'StartDate': '7/17/18',
  'Subgenre': 'Pop',
  'Venue': '"FirstEnergy Stadium, Home of the Cleveland Browns"'},
 {'Latitude': 41.542609399999996,
  'Longitude': -87.7711753,
  'Name': 'Ozzy Osbourne: No More Tours 2',
  'PriceMaximum': 250.0,
  'PriceMinimum': 29.5,
  'StartDate': '9/21/18',
  'Subgenre': 'Heavy Metal',
  'Venue': '"Hollywood Casino Amphitheatre - Chicago, IL"'},
 {'Latitude': 41.542609399999996,
  'Longitude': -87.7711753,
  'Name': 'Lynyrd Skynyrd: Last of the Street Survivors Farewell Tour',
  'PriceMaximum': 199.5,
  'PriceMinimum': 29.5,
  'StartDate': '8/3/18',
  'Subgenre': 'Pop',
  'Venue': '"Hollywood Casino Amphitheatre - Chicago, IL"'},
 {'Latitude': 41.542609399999996,
  'Longitude': -87.7711753,
  'Name': 'Imagine Dragons: EVOLVE TOUR',
  'PriceMaximum': 146.0,
  'PriceMinimum': 35.0,
  'StartDate': '7

In [ ]:
result_samples_desc = session.query(samples_db.otu_id,getattr(samples_db, s)).order_by(getattr(samples_db, s).desc()).all()
result_samples_desc

In [28]:
url="/maps/StartDate/7/7/18"
criteria=url.split("/")
print(criteria[2])
d=criteria[3]+"/"+criteria[4]+"/"+criteria[5]
print(d)
result_wfreq= session.query(events_db.Latitude,events_db.Longitude,events_db.Name,events_db.PriceMaximum,\
                       events_db.PriceMinimum,events_db.StartDate,events_db.Subgenre,events_db.Venue).filter(getattr(events_db,criteria[2])==d).all()
result_wfreq

StartDate
7/7/18


[(41.542609399999996,
  -87.7711753,
  'Weezer / Pixies',
  93.0,
  25.0,
  '7/7/18',
  'Alternative Rock',
  '"Hollywood Casino Amphitheatre - Chicago, IL"'),
 (38.7456419,
  -90.46995600000001,
  'Evanescence + Lindsey Stirling',
  108.5,
  25.0,
  '7/7/18',
  'Alternative Rock',
  '"Hollywood Casino Amphitheatre - St. Louis, MO"'),
 (47.596703999999995,
  -122.332085,
  'Kenny Chesney: Trip Around the Sun Tour',
  275.0,
  29.0,
  '7/7/18',
  'Country',
  'CenturyLink Field'),
 (34.07359365,
  -118.2399223,
  'Dead & Company',
  174.5,
  45.0,
  '7/7/18',
  'Pop',
  'Dodger Stadium'),
 (40.419781799999996,
  -80.4342124,
  'Jimmy Buffett',
  146.0,
  36.0,
  '7/7/18',
  'Pop',
  'KeyBank Pavilion'),
 (33.9942619,
  -81.03610400000001,
  'Imagine Dragons: EVOLVE TOUR',
  129.5,
  29.5,
  '7/7/18',
  'Pop',
  'Colonial Life Arena'),
 (36.159949,
  -86.77857,
  'Sam Smith: The Thrill Of It All Tour',
  120.5,
  31.35,
  '7/7/18',
  'Pop',
  'Bridgestone Arena'),
 (36.102916,
  -115.169